# Image flip using OpenCV 
### [(Example using phase contrast microsocopy data)](https://www.kaggle.com/c/sartorius-cell-instance-segmentation)
## Author: [Dr. Rahul Remanan](https://www.linkedin.com/in/rahulremanan/)
## CEO, [Moad Computer](https://moad.computer)
In this notebook, the input images using the phase contrast microscopy images and their corresponding mask labels, from the [Sartorius cell instance segmentation challenge data-set in Kaggle](https://www.kaggle.com/c/sartorius-cell-instance-segmentation), are horizontally and vertically flipped using [OpenCV](https://opencv.org/).

# Import libraries

In [ ]:
import cv2, math, random, numpy as np, pandas as pd, \
       tensorflow as tf, matplotlib.pyplot as plt
from skimage import io as imageIO

# Flip an image using OpenCV

In [ ]:
def flip(img, axis):
  return cv2.flip(img, axis)

def flip_masks(masks, axis): 
  if len(masks.shape) == 3:
    out_masks = list(map(lambda i: cv2.flip(masks[:,:,i], axis), range(masks.shape[-1])))
    out_masks = np.moveaxis(out_masks, 0, 2)
  else:
    out_masks = cv2.flip(masks, axis)
  return out_masks

# Demo of cv2.flip()

## Read an example image and perform horizontal and vertical flipping using OpenCV

In [ ]:
img = imageIO.imread('https://github.githubassets.com/images/modules/logos_page/Octocat.png')
vflip_img, hflip_img = flip(img, 0), flip(img, 1)

## Plot original image and flipped images

In [ ]:
plt.figure(figsize=(15, 15))

plt.subplot(1, 3, 1)
plt.imshow(img,cmap='gray')
plt.title('Image') 
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(vflip_img,cmap='gray')
plt.title('Vertically flipped image') 
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(hflip_img,cmap='gray')
plt.title('Horizontally flipped image') 
plt.axis('off')

plt.show()

# Initialize variables

In [ ]:
ROOT_DIR = '/kaggle/input'
DATA_DIR = f'{ROOT_DIR}/sartorius-cell-instance-segmentation'
TRAIN_CSV = f'{DATA_DIR}/train.csv'

INPUT_HEIGHT              = 520
INPUT_WIDTH               = 704
IMAGE_HEIGHT              = 576
IMAGE_WIDTH               = 704
INPUT_CHANNELS            = 3
IMAGE_CHANNELS            = 3
INPUT_SIZE                = [INPUT_HEIGHT, INPUT_WIDTH]
IMAGE_SIZE                = [IMAGE_HEIGHT, IMAGE_WIDTH]

# Read phase-contrast microscopy images and mask files

In [ ]:
df = pd.read_csv(TRAIN_CSV)
display(df.cell_type.unique())
display(df.cell_type.hist())

# Helper functions

In [ ]:
#ref: https://www.kaggle.com/inversion/run-length-decoding-quick-start
def rle_decode(mask_rle, mask_size, color=1):
  '''
  mask_rle: run-length as string formated (start length)
  shape: (height, width, channels) of array to return 
  color: color for the mask
  Returns numpy array (mask)
  '''
  s = mask_rle.split()
  starts = list(map(lambda x: int(x) - 1, s[0::2]))
  lengths = list(map(int, s[1::2]))
  ends = [x + y for x, y in zip(starts, lengths)]
  mask = np.full((mask_size[0], mask_size[1]), 0)
  img = mask.reshape((mask_size[0] * mask_size[1]))
  for start, end in zip(starts, ends):
    img[start : end] = color
  return img.reshape(mask_size)

def read_mask(imgID, img_size):
  num_rows = len(df[df.id == imgID])  
  masks = list(map(lambda i: rle_decode(df[df.id == imgID].annotation.iloc[i], img_size), 
              range(num_rows)))
  masks = np.array(masks)
  masks = np.moveaxis(masks, 0, 2)
  return masks

def read_png(filename):
  img   = tf.io.read_file(filename)
  img   = tf.image.decode_png(img, channels=INPUT_CHANNELS)
  img   = tf.expand_dims(img, -1)
  img   = tf.cast(img, tf.float32)
  img   = img / 255.
  img   = tf.reshape(img, [*INPUT_SIZE, INPUT_CHANNELS])
  return img

def resize_masks(masks, shape): 
  if len(masks.shape) == 3:
    print(masks.shape)
    print(shape)
    out_masks = list(map(lambda i: cv2.resize(masks[:,:,i].astype(np.uint8), shape), 
                         range(masks.shape[-1])))
    out_masks = np.array(out_masks).reshape([masks.shape[-1], 
                                             shape[1], shape[0]])
    out_masks = np.moveaxis(out_masks, 0, 2) 
  else:
    out_masks = cv2.resize(masks, shape)
  return np.array(out_masks)

# Display image and its corresponding mask

In [ ]:
def display_masked_image(img, mask):  
  if len(img.shape)==3:
    img   = img[...,0]
  dim   = img.shape
  img   = cv2.resize(img, (dim[0], dim[1]))
  mask  = cv2.resize(mask, (dim[0], dim[1]))
    
  plt.figure(figsize=(15, 15))
  plt.subplot(1, 3, 1)
  plt.imshow(img,cmap='gray')
  plt.title('Image') 
  plt.axis('off')
   
  plt.subplot(1, 3, 2)
  plt.imshow(mask, cmap='inferno')
  plt.title('Mask')
  plt.axis('off')
 
  plt.subplot(1, 3, 3)
  plt.imshow(img)
  plt.imshow(mask, alpha=0.4, cmap='inferno')
  plt.title('Image + Mask')
  plt.axis('off')
  plt.tight_layout()
  plt.show()

# Flip an example image and its corresponding masks

## Read an example image and its masks

In [ ]:
split = 'train'
img_files = tf.io.gfile.glob(f'{DATA_DIR}/{split}/*.png')
img_file = random.SystemRandom().choice(img_files)
img_id = img_file.split('/')[-1].replace('.png', '')
print(f'Reading image file id: {img_id} ...')
img = read_png(img_file).numpy()
masks = read_mask(img_id, tuple(INPUT_SIZE))
img = cv2.resize(img, tuple(IMAGE_SIZE))
masks = resize_masks(masks, tuple(IMAGE_SIZE))
print(img.shape, masks.shape)

## Horizontally and vertically flip the image-masks pairs

In [ ]:
hflip_img, hflip_masks = flip(img, 1), flip_masks(masks, 1)
vflip_img, vflip_masks = flip(img, 0), flip_masks(masks, 0)
print(hflip_img.shape, hflip_masks.shape)
print(vflip_img.shape, vflip_masks.shape)

# Display the input image, its mask and the flipped output

In [ ]:
print('='*10,'Original image and mask','='*10)
display(display_masked_image(img, 
                             np.clip(np.sum(masks, axis=-1), 0,1).astype(np.uint8)))
print('='*10,'Horizontally flipped image and mask','='*10)
display(display_masked_image(hflip_img, 
                             np.clip(np.sum(hflip_masks, axis=-1).astype(np.uint8), 0,1)))
print('='*10,'Vertically flipped image and mask','='*10)
display(display_masked_image(vflip_img, 
                             np.clip(np.sum(vflip_masks, axis=-1).astype(np.uint8), 0,1)))

### References:
1. [OpenCV operations on arrays to flip images](https://docs.opencv.org/4.5.4/d2/de8/group__core__array.html#gaca7be533e3dac7feb70fc60635adf441)